# Data Augmentation

Neste trabalho, utilizamos técnicas de Data Augmentation para ampliar o dataset original composto por imagens de gráficos de barras extraídas de documentos. O código desenvolvido realiza as seguintes etapas: (1) organiza o dataset em subpastas baseadas nas classes ("bars" e "no"), preservando as imagens originais; (2) aplica transformações específicas de aumento de dados para simular variações realistas nos gráficos, como translações, alterações de escala, ajustes de brilho, contraste, saturação e recortes aleatórios; e (3) gera novas imagens aumentadas com base em um fator de aumento definido (por exemplo, triplicar o dataset). As transformações foram cuidadosamente projetadas para refletir possíveis variações nos gráficos encontrados em diferentes documentos, enquanto preservam as características essenciais para a tarefa de classificação. Ao final, o dataset aumentado é salvo em um novo diretório, estruturado de forma organizada para facilitar o treinamento de modelos de aprendizado profundo.

As imagens originais foram retirades de: https://www.kaggle.com/datasets/sunedition/graphs-dataset

In [ ]:
import os
from torchvision import transforms
from PIL import Image
import random

# Configurações
base_dir = "..."  # Pasta com subpastas 'bars' e 'no bars'
augmented_dir = "..."  # Onde salvar o dataset aumentado
augmentation_factor = 3  # Fator de aumento (2x = dobrar, 3x = triplicar)
img_size = 128

# Transformações específicas para gráficos de barras
augmentation_transforms = transforms.Compose([
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.8, 1.2)),  # Escala e deslocamento
    transforms.RandomResizedCrop(size=img_size, scale=(0.8, 1.0)),  # Simula gráficos incompletos
    transforms.Resize((img_size, img_size)),  # Redimensiona apenas versões aumentadas
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Ajustes de cor
    transforms.ToTensor(),  # Converte para tensor (necessário antes de Normalize)
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalização
])

# Criar diretório para o dataset aumentado
if not os.path.exists(augmented_dir):
    os.makedirs(augmented_dir)

# Função para aplicar Data Augmentation em uma classe
def augment_class(class_name, num_augmentations):
    class_dir = os.path.join(base_dir, class_name)
    output_dir = os.path.join(augmented_dir, class_name)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Filtrar apenas imagens válidas
    valid_extensions = ['.jpg', '.jpeg', '.png']
    original_images = [img for img in os.listdir(class_dir) 
                       if os.path.isfile(os.path.join(class_dir, img)) and img.lower().endswith(tuple(valid_extensions))]
    
    for img_name in original_images:
        # Copiar imagem original para o dataset aumentado
        original_path = os.path.join(class_dir, img_name)
        Image.open(original_path).save(os.path.join(output_dir, img_name))
    
    # Gerar imagens aumentadas
    for i in range(num_augmentations):
        img_name = random.choice(original_images)
        img_path = os.path.join(class_dir, img_name)
        image = Image.open(img_path).convert("RGB")
        augmented_image = augmentation_transforms(image)
        
        # Salvar imagem aumentada
        augmented_image_pil = transforms.ToPILImage()(augmented_image)
        augmented_image_pil.save(os.path.join(output_dir, f"aug_{i}.jpg"))

# Aplicar Data Augmentation para cada classe
for class_name in ['bars', 'no bars']:
    class_dir = os.path.join(base_dir, class_name)
    num_original_images = len([f for f in os.listdir(class_dir) 
                                if os.path.isfile(os.path.join(class_dir, f)) and not f.startswith('.ipynb_checkpoints')])
    num_augmentations = num_original_images * (augmentation_factor - 1)  # Dobrar ou triplicar
    print(f"Aumentando classe '{class_name}' de {num_original_images} para {num_original_images + num_augmentations} imagens...")
    augment_class(class_name, num_augmentations)

print(f"Dataset aumentado salvo em {augmented_dir}")
